In [1]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:45:30_PST_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0


In [2]:
!pip install --extra-index-url=https://pypi.nvidia.com polars cudf-polars-cu11

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [3]:
import polars as pl
import json
from pathlib import Path
import gc

def transform_and_save_to_parquet(file, output_dir):
    # LazyFrame으로 NDJSON 파일 읽기
    df = pl.scan_ndjson(file)
        
    # 각 세션의 이벤트를 분리하여 2차원 배열 구조로 변환
    expanded_df = df.explode("events").select(
        pl.col("session"),
        pl.col("events").struct.field("aid").alias("aid"),
        pl.col("events").struct.field("ts").alias("ts"),
        pl.col("events").struct.field("type").alias("type")
    )
    
    # type을 0, 1, 2, 3으로 치환
    expanded_df = expanded_df.with_columns(
        pl.when(pl.col("type") == "clicks").then(0)
        .when(pl.col("type") == "carts").then(1)
        .when(pl.col("type") == "orders").then(2)
        .otherwise(3).alias("type")
    )
    
    # 데이터 타입 최적화
    expanded_df = expanded_df.with_columns([
        pl.col("session").cast(pl.UInt32),
        pl.col("aid").cast(pl.UInt32),
        (pl.col("ts") // 1000).cast(pl.UInt32),     # timestamp를 초 단위로 줄임 (밀리초 -> 초 변환)
        pl.col("type").cast(pl.UInt8)
    ])
    
    # LazyFrame 병합 후 Parquet 파일로 저장
    combined_df = expanded_df.collect()  # 데이터 수집
    combined_df.write_parquet(f"{output_dir}/train.parquet", compression="snappy")
    
    # 메모리 정리
    del combined_df
    gc.collect()

# NDJSON 파일을 읽어오기
file = Path("otto-recsys-train.jsonl")

# Parquet 파일로 저장할 디렉토리 설정
output_dir = "working"
Path(output_dir).mkdir(exist_ok=True)

# Parquet 파일로 저장
%time transform_and_save_to_parquet(file, output_dir)

CPU times: user 2min 49s, sys: 1min 12s, total: 4min 1s
Wall time: 41.6 s


In [4]:
train_df = pl.read_parquet("working/train.parquet")

In [5]:
train = train_df.to_pandas()
train.head()


,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0
